In this notebook, we show code for estimating the multinomial mixture algorithm using the EM algorithm.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import topicmodels  # pip install topic-modelling-tools

For the illustration, we use text data from US State-of-the-Union addreses accessed from Github.

In [2]:
url = "https://github.com/sekhansen/text-mining-tutorial/raw/master/speech_data_extend.txt"
df = pd.read_table(url, encoding="utf-8")
df.head()

,president,speech,year
0,Washington,Fellow-Citizens of the Senate and House of Rep...,1790
1,Washington,I embrace with great satisfaction the opportun...,1790
2,Washington,In resuming your consultations for the general...,1790
3,Washington,Among the many interesting objects which will ...,1790
4,Washington,"A free people ought not only to be armed, but ...",1790


We only keep data from the year 2000 to speed computation.

In [3]:
df = df[df.year >= 2000]
print(df.shape)
df.head()

(1232, 3)


,president,speech,year
21923,Clinton,"Mr. Speaker, Mr. Vice President, Members of Co...",2000
21924,Clinton,We are fortunate to be alive at this moment in...,2000
21925,Clinton,We begin the new century with over 20 million ...,2000
21926,Clinton,And our economic revolution has been matched b...,2000
21927,Clinton,"My fellow Americans, the state of our Union is...",2000


There are 1,232 paragraphs in the data.  Here we define a document as a paragraph of a State-of-the-Union speech.

We perform the standard pre-processing steps discussed in the information retrieval notebook.

In [4]:
docsobj = topicmodels.RawDocs(df.speech.values, "long")
docsobj.token_clean(1)
docsobj.stopword_remove("tokens")
docsobj.stem()
docsobj.stopword_remove("stems")
docsobj.term_rank("stems")
docsobj.rank_remove("tfidf", "stems", docsobj.tfidf_ranking[2300][1])

Next we form the bag-of-words for the data, and store the mapping from term indices to terms.

In [5]:
bowobj = topicmodels.BOW(docsobj.stems)
V = bowobj.bow.shape[1] # dimensionality of document-term matrix
print(V)
inv_map = {v: k for k, v in bowobj.token_key.items()}
print(inv_map[10]) # illustrative token (here the 11th token in the vocabulary)

2119
risk


Next we estimate the multinomial mixture model with $K=2$ starting from $5$ different random initial values for the parameters, and store the estimated model with the high log-likelihood achieved after convergence.

In [6]:
K=2

emobjs = []
loglik = []

for i in range(5):
    emobj = topicmodels.multimix.EM(bowobj.bow, K) # pass the document-term matrix and K=2
    emobj.estimate(1000) # 1000 is maximum number of iterations
    loglik.append(emobj.loglik[-1])
    emobjs.append(emobj)
    print("Iteration", i+1)

em_best = emobjs[np.argmax(loglik)]

convergence at iteration 58, loglik = -259978.232300
Iteration 1
convergence at iteration 28, loglik = -260263.978654
Iteration 2
convergence at iteration 53, loglik = -260438.082410
Iteration 3
convergence at iteration 111, loglik = -260910.504415
Iteration 4
convergence at iteration 408, loglik = -261505.333250
Iteration 5


First we can observe the log-likelihood of the best-fit model at each iteration.  Recall the EM algorithm will increase the value of the log-likelihood at each iteration.

In [7]:
em_best.loglik

array([-312800.41304647, -264775.6371536 , -263275.08287791,
       -261468.31962337, -260612.50709653, -260329.35055897,
       -260180.46226923, -260089.36617841, -260054.63660178,
       -260033.83929252, -260011.142693  , -260002.38501813,
       -260000.28186984, -259993.52707618, -259992.10603947,
       -259991.9089792 , -259991.5272684 , -259991.25509288,
       -259990.96147203, -259989.21364744, -259987.27901749,
       -259987.18365775, -259987.13416613, -259987.1110509 ,
       -259987.10213481, -259987.09899946, -259987.09793535,
       -259987.09757854, -259987.09745933, -259987.09741948,
       -259987.09740591, -259987.09740029, -259987.09739406,
       -259987.09737457, -259987.09729854, -259987.09699514,
       -259987.09577518, -259987.0907563 , -259987.06816972,
       -259986.92567825, -259985.12238309, -259978.68856164,
       -259978.25400745, -259978.24092477, -259978.23592499,
       -259978.23386481, -259978.23298532, -259978.23260274,
       -259978.23243447,

Next, we are interested in the estimated parameters.

In [8]:
em_best.rho # the estimated mixing probabilities over the two topics

array([0.58107194, 0.41892806])

In addition to the estimated mixing probabilities, the model estimates the probability that each term appears in each of the topics.  We construct a matrix that orders terms based on their probability in each topic.

In [9]:
indices = np.zeros((K, V), dtype=np.int)
for k in range(K):
    indices[k, :] = em_best.mu[k, :].argsort()

top = [[] for k in range(K)]

for i in range(-1, -20, -1):
    for k in range(K):
        top[k].append(inv_map[indices[k][i]])

In [10]:
print(top[0]) # top terms in topic0

[u'job', u'help', u'tax', u'need', u'busi', u'must', u'health', u'economi', u'congress', u'care', u'energi', u'school', u'ask', u'time', u'million', u'let', u'tonight', u'reform', u'children']


In [11]:
print(top[1]) # top terms in topic1

[u'world', u'countri', u'must', u'terrorist', u'iraq', u'state', u'secur', u'freedom', u'govern', u'unit', u'time', u'come', u'war', u'weapon', u'iraqi', u'peac', u'tonight', u'know', u'support']


Here it appears the model has uncovered an interpretable latent space with a "domestic policy" dimension and a "foreign policy" dimension.